逆問題を解いてみよう
===
by ほげにむし

逆問題の例として、以下のような問題を考えてみよう。


問題
---
---
9個のセルが以下のように並んでいる。9個のセルの中身を直接見ることはできないが、
セルの集まりの、水平、垂直、対角線方向、の和を求めることはできるとする。


$$ \begin{array}{ccc} x_1 x_2 x_3 \\x_4 x_5 x_6 \\x_7 x_8 x_9\\ \end{array}  $$


この和は全部で16個あるが、
- いくつかが読み取れない
- ノイズが乗っている

場合がある。

この時に、個々のセルの値に近い値を求めることができるだろうか？
できるとすれば、どの程度正しい値が得られるだろうか？

---

この問題を解くために、状況を少し整理しよう。
周辺から観測したセルの和を$b_i$とすると、セル$x_j$との関係は、以下のようになる。


１．左から右にセルを通過する場合

$b_1=x_1+x_2+x_3$

$b_2=x_4+x_5+x_6$

$b_3=x_7+x_8+x_9$

２．右上から左下にセルを通過する場合

$b_4=x_1$

$b_5=x_2 + x_4$

$b_6=x_3 + x_5 + x_7$

$b_7=x_6 + x_8$

$b_8=x_9$

３．上から下にセルを通過する場合

$b_9=x_1 + x_4 + x_7$

$b_{10}=x_2 + x_5 + x_8$

$b_{11}=x_3 + x_6 + x_9$

４．左上から右下にセルを通過する場合

$b_{12}=x_7$

$b_{13}=x_4 + x_8$

$b_{14}=x_1 + x_5 + x_9$

$b_{15}=x_2 + x_6$

$b_{16}=x_3$

${\bf x}\in {\bf R}^9$、 ${\bf b}\in {\bf R}^{16}$、
${\bf A}\in {\bf R}^{16\times 9}$として、これに対応する行列を作ろう
今回は記号演算の必要がないのでNumPyを用いる。

まずは係数の行列をつくる。今回は数が少ないので力業で作ろう。

In [80]:
import numpy as np
A = np.array([[1.0,1,1,0,0,0,0,0,0],
                [0,0,0,1,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,1],
                [1,0,0,0,0,0,0,0,0],
                [0,1,0,1,0,0,0,0,0],
                [1,0,0,0,1,0,0,0,1],
                [0,0,0,0,1,0,0,0,1],
                [0,0,0,0,0,0,0,0,1],
                [1,0,0,1,0,0,1,0,0],
                [0,1,0,0,1,0,0,1,0],
                [0,0,1,0,0,1,0,0,1],
                [0,0,0,0,0,0,1,0,0],
                [0,0,0,1,0,0,0,1,0],
                [1,0,0,0,1,0,0,0,1],
                [0,1,0,0,0,1,0,0,0],
                [0,0,1,0,0,0,0,0,0]])
A

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

思った通り大変である。

$m\times n$の一般の長方形形状のセルの集合は、課題としてとっておこう。

さて、係数が求まったので、一般逆行列を求めることができる。
これはNumPyのpinv関数を使う。


In [81]:
Ainv=np.linalg.pinv(A)

では、セルを
$$
\begin{array}{ccc}
1 2 3 \\
4 5 6 \\
7 8 9
\end{array}
$$
と考えて、答えのベクトル${\bf x}$を作ろう。

何の変哲もない並びだが、確認にはよかろう。

In [82]:
x = np.array([1.0, 2, 3, 4, 5, 6, 7, 8, 9])
x

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

このセル群を縦、横、斜めから見た時の和を計算するにはどうするのだろうか？

先ほど定義した行列は${\bf x}$ が入力されたときに和の取り方を示したものなので、
単純に掛け算をすればよいはずである。

行列とベクトル、ベクトルとベクトルの積は、dot関数で行う。実際にやってみよう。


In [84]:
b=A.dot(x)
b

array([  6.,  15.,  24.,   1.,   6.,  15.,  14.,   9.,  12.,  15.,  18.,
         7.,  12.,  15.,   8.,   3.])

出てきた。升目を見ながら加算を行い、このリストと見比べると、結果が意図したものと一致することがわかる。

つまり、${\bf Ax}={\bf b}$のすべての要素がわかった。

次は${\bf x}$を未知とする。Aの疑似逆行列を求めて、上式の左からかけよう。
疑似逆行列は、NumPyのpinv関数で求めることができる。

In [87]:
Ainv=np.linalg.pinv(A)
Ainv.dot(b)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

元に戻った！

次に、${\bf b}$に少しノイズを加えてみよう。

In [92]:
c=np.array(b)
c[0] += 1
Ainv.dot(c)

array([ 1.1529052 ,  2.19397117,  3.27435561,  3.95587593,  4.96941896,
        5.88816077,  6.98864133,  7.98339886,  8.94189602])

一つ値を変えただけで、全体の値が少しずつ変わってしまうことがわかる。

今度は異なる部分を変えてみよう。


In [94]:
d=np.array(b)
d[3]+=1
Ainv.dot(d)

array([ 1.37003058,  1.91131498,  2.87155963,  3.91131498,  4.85932722,
        6.17125382,  6.87155963,  8.17125382,  8.93272171])

In [78]:
def magic_square3(coef, b):
    A = np.array(coef)
    s = np.array(b)
    for i in reversed(range(b.shape[0])):
        if(np.isnan(b[i])):
            np.delete(A, i, 0)
            s=np.delete(s,i)
    return A,s

In [75]:
magic_square3(A,b)

(array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 array([ 18.,  23.,   4.,   5.,  22.,   4.,  -1.,  -5.,  22.,  18.,   5.,
          7.,  12.,   4.,  21.,   1.]))

In [76]:
c=np.array(b)
c[2]=np.nan
c[4]=np.nan
c[6]=np.nan
c

array([ 18.,  23.,  nan,   5.,  nan,   4.,  nan,  -5.,  22.,  18.,   5.,
         7.,  12.,   4.,  21.,   1.])

In [79]:
magic_square3(A,c)

(array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 array([ 18.,  23.,   5.,   4.,  -5.,  22.,  18.,   5.,   7.,  12.,   4.,
         21.,   1.]))